In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import random
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import DataLoader, SubsetRandomSampler

In [3]:
# mydrive_path = '/home/ramineon/Proposal/Test_Thesis_Dataset/data'
mydrive_path = '/home/ramineon/Proposal/Thesis1/data'

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Define the transformations for both training and test data
transform = transforms.Compose([
    transforms.Resize((224, 224)),       # Resize images to 224x224 for VGG16
    transforms.ToTensor(),               # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])  # Normalize for pre-trained models
])
patheon = mydrive_path
# Load the dataset
dataset = datasets.ImageFolder(root=patheon, transform=transform)

# Define the train/test split ratio
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size

# Randomly split the dataset
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Define DataLoaders for both training and test sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer

class ImprovedCNNWithTransformer(nn.Module):
    def __init__(self, num_classes=2, num_transformer_layers=4, num_heads=8):
        super().__init__()
        
        # Convolutional Layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        
        self.shortcut = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=1, stride=2),
            nn.BatchNorm2d(128)
        )
        
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        
        # Transformer Parameters
        self.embed_dim = 128  # Token embedding size
        self.global_pool = nn.AdaptiveAvgPool2d((4, 4))  # Fixed-size output for tokenization

        # Transformer Encoder
        encoder_layer = TransformerEncoderLayer(d_model=self.embed_dim, nhead=num_heads, dim_feedforward=512, dropout=0.4)
        self.transformer = TransformerEncoder(encoder_layer, num_layers=num_transformer_layers)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(self.embed_dim, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        # Block 1
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)

        # Block 2
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)

        # Block 3 with Residual
        shortcut = self.shortcut(x)  # Downsample shortcut
        x = F.relu(self.bn3(self.conv3(x)) + shortcut)
        
        # Global Pooling
        x = self.global_pool(x)  # Shape: [batch_size, 128, 4, 4]
        batch_size, channels, height, width = x.size()

        # Prepare Transformer Input
        x = x.view(batch_size, channels, -1).permute(0, 2, 1)  # Shape: [batch, 16, 128]
        
        # Transformer Encoder
        x = self.transformer(x)  # Shape: [batch, 16, 128]
        x = x.mean(dim=1)  # Aggregate token representations (Shape: [batch, 128])

        # Fully Connected Layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x
model = ImprovedCNNWithTransformer()

/home/ramineon/ThesisEon/venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [69]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()

# Updated optimizer with weight decay
# optimizer = optim.Adam(model.parameters(), lr=3e-4)
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-2)
num_epochs = 1  # Adjust as needed
model.to(device)  # Ensure the model is moved to the device

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # Training loop
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to device

        # Forward pass and optimization
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Track loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    # Print detailed epoch metrics
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_samples
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, "
          f"Accuracy: {epoch_accuracy:.2%}, "
          f"LR: {optimizer.param_groups[0]['lr']:.6f}")

# Evaluation loop
model.eval()
correct_predictions = 0
total_samples = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to device
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

# Print test accuracy
test_accuracy = correct_predictions / total_samples
print(f"Test Accuracy: {test_accuracy:.2%}")


Epoch [1/1], Loss: 0.0502, Accuracy: 98.15%, LR: 0.000300
Test Accuracy: 89.74%


In [65]:
"""
Test section

Explanation of Metrics
Overall Test Accuracy:

Direct comparison of predicted vs. actual labels across the entire test set.
accuracy_score(all_labels, all_preds) calculates this metric.
Per-Class Accuracy:

Measures the accuracy of the model for each individual class.
Computed using the diagonal values of the confusion matrix (correct predictions per class) divided by the number of samples per class.
Classification Report:

Includes precision, recall, and F1-score for each class.
This provides a detailed understanding of how well the model performs for each class, especially in cases of class imbalance.
Confusion Matrix:

Shows the number of true positives, false positives, false negatives, and true negatives for each class.
Useful for diagnosing specific patterns of misclassification.

"""

import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            # Get predictions
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Convert to numpy arrays for evaluation
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Overall accuracy
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Overall Test Accuracy: {accuracy:.4f}")

    # Per-class accuracy
    class_accuracy = np.diag(confusion_matrix(all_labels, all_preds)) / np.bincount(all_labels)
    for i, acc in enumerate(class_accuracy):
        print(f"Class {i} Accuracy: {acc:.4f}")

    # Classification report
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=dataset.classes))

    # Confusion matrix
    print("\nConfusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

    return accuracy

# Test the model
test_accuracy = evaluate_model(model, test_loader, device)

Evaluating: 100%|██████████| 5/5 [00:00<00:00,  6.24it/s]

Overall Test Accuracy: 0.9167
Class 0 Accuracy: 0.9211
Class 1 Accuracy: 0.9125

Classification Report:
              precision    recall  f1-score   support

           1       0.91      0.92      0.92        76
           2       0.92      0.91      0.92        80

    accuracy                           0.92       156
   macro avg       0.92      0.92      0.92       156
weighted avg       0.92      0.92      0.92       156


Confusion Matrix:
[[70  6]
 [ 7 73]]


In [70]:
torch.save(model.state_dict(), "model_weights.pth")

In [71]:
torch.save(model, "model_complete.pth")


In [ ]:
"""
# Loading a Saved Model
# Load Only Model Parameters:
# First, recreate the model architecture, then load the saved weights.
# python
# Copy code
# Recreate the model architecture
model = ImprovedCNNWithTransformer(num_classes=2)  # Ensure the architecture matches

# Load the weights
model.load_state_dict(torch.load("model_weights.pth"))

# Switch to evaluation mode
model.eval()
# Load the Entire Model:
# Directly load the saved model.
# python
# Copy code
model = torch.load("model_complete.pth")
model.eval()
"""

In [ ]:
"""
If you save the model on one device (e.g., GPU) and load it on another (e.g., CPU), use map_location when loading:
"""
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss
}, "checkpoint.pth")

"""
Saving Optimizer State (Optional):
If you want to resume training later, save the optimizer's state too:
"""
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss
}, "checkpoint.pth")

"""
To load the checkpoint:
"""
checkpoint = torch.load("checkpoint.pth")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']


In [72]:
import os
print(os.getcwd())  # Prints the current working directory


/home/ramineon/Proposal/Thesis ipynb
